In [4]:
import sys
sys.path.append("/mnt/ssd500/hungbnt/transformer-deploy/src")
from transformer_deploy.backends.pytorch_utils import convert_to_onnx
from transformers import AutoModelForTokenClassification, AutoTokenizer


ModuleNotFoundError: No module named 'onnxoptimizer'

In [2]:
# Download the fine-tuned model from W&B artifacts
import wandb


api = wandb.Api()
artifact = api.artifact("int_pb/hf-end-to-end-deployment/model-1lv0fpmm:v0")
path = artifact.download()


wandb: Downloading large artifact model-1lv0fpmm:v0, 256.35MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:53.0


In [ ]:
!docker run -it --rm --gpus all \
  -v $PWD:/project ghcr.io/els-rd/transformer-deploy:0.4.0 \
  bash -c "cd /project && \
    convert_model -m \"philschmid/MiniLM-L6-H384-uncased-sst2\" \
    --backend tensorrt onnx \
    --seq-len 16 128 128"

# output:  
# ...
# Inference done on NVIDIA GeForce RTX 3090
# latencies:
# [Pytorch (FP32)] mean=5.43ms, sd=0.70ms, min=4.88ms, max=7.81ms, median=5.09ms, 95p=7.01ms, 99p=7.53ms
# [Pytorch (FP16)] mean=6.55ms, sd=1.00ms, min=5.75ms, max=10.38ms, median=6.01ms, 95p=8.57ms, 99p=9.21ms
# [TensorRT (FP16)] mean=0.53ms, sd=0.03ms, min=0.49ms, max=0.61ms, median=0.52ms, 95p=0.57ms, 99p=0.58ms
# [ONNX Runtime (FP32)] mean=1.57ms, sd=0.05ms, min=1.49ms, max=1.90ms, median=1.57ms, 95p=1.63ms, 99p=1.76ms
# [ONNX Runtime (optimized)] mean=0.90ms, sd=0.03ms, min=0.88ms, max=1.23ms, median=0.89ms, 95p=0.95ms, 99p=0.97ms
# Each infence engine output is within 0.3 tolerance compared to Pytorch output


In [ ]:
docker run -it --rm --gpus 1 \
  -v $PWD:/project ghcr.io/els-rd/transformer-deploy:0.5.3 \
  bash -c "cd /project && \
    convert_model -m artifacts/model-1lv0fpmm:v0 \
    --backend tensorrt onnx \
    --seq-len 16 128 128"
